In [ ]:
import pandas as pd
import numpy as np
import json
import toml
import math
import matplotlib.pyplot as plt
import subprocess
%matplotlib notebook

# Pathfinder Visualization
This uses our Java robot code and the [Pathfinder](https://github.com/JacisNonsense/Pathfinder) library to output trajectory points to a `.csv` file we use to plot.

Parameters are:

- `dt` = iteration time, loop period (ms)
- `v_max` = max velocity (ft/sec)
- `a_max` = max acceleration (ft/sec/sec)
- `j_max` = max jerk (ft/sec/sec/sec)
- `waypoints` = x, y, and exit angle (degrees) 


In [ ]:
toml_str = """
waypoints = [
  {x = -4.0, y = -1.0, angle = -45.0},
  {x = -2.0, y = -2.0, angle = 0.0},
  {x = 0.0, y = 0.0, angle = 0.0},
]
fit              = "HERMITE_CUBIC"
sample_count     = 100_000
dt               = 0.050    # sec
max_velocity     = 1.7   # m/sec
max_acceleration = 2.0  # m/sec/sec
max_jerk         = 60.0  # m/sec/sec/sec
"""

In [ ]:
process = subprocess.run(
    "MOTION_OPTS=-Djava.library.path={0}/lib {0}/bin/motion '{1}'".format(
        "../build/install/motion",
        json.dumps({"pathfinder": {"toml": toml_str}})),
    shell=True)

df = pd.read_csv("trajectory.csv")
path = toml.loads(toml_str)
wp = pd.DataFrame(path['waypoints'])

In [ ]:
print("time to end point    = {:.0f} ms".format(df.index.max()))
print("max velocity         = {:.1f} ft/sec".format(df['velocity'].max()))
print("max acceleration     = {:.1f} ft/sec^2".format(df['acceleration'].max()))
print("max jerk             = {:.1f} ft/sec^3".format(df['jerk'].max()))

In [ ]:
ax = df.plot.scatter('x', 'y')
wp.plot.scatter('x', 'y', c='r', ax=ax)
ax.axis('equal')
plt.grid(True, color='lightgray', linestyle='--', linewidth=0.6)

In [ ]:
df.loc[:,['position', 'velocity', 'acceleration']].plot()
plt.grid(True, color='lightgray', linestyle='--', linewidth=0.6)
plt.xlabel("Time (ms)")
l = plt.legend(["Position ($m$)", "Velocity ($m/sec$)", "Acceleration ($m/sec^2$)"])